In [ ]:
#Lattice Boltzman D3Q15 Simulation of Swimmers with Search
#Disease Biophysics Group
#Written by John Zimmerman
#Updated 9/08/22

%matplotlib inline
import tensorflow as tf
import os
import time
import numpy as np
import math 
import cv2
import pandas as pd
import h5py
from tqdm import *
from scipy.spatial import ConvexHull
from itertools import product
import scipy.stats

import GeoSwimmer
import DBG_LBM
import SwimMesh
import SwimNN
from tensorflow.python.framework.ops import disable_eager_execution
from scipy.interpolate import UnivariateSpline

from matplotlib.pyplot import *
%matplotlib inline
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

print(tf.__version__)
tf.compat.v2.keras.backend.clear_session()

In [ ]:
def binarySign(xarray):
    xarray[xarray>0] = 1.0
    xarray[xarray<0] = -1.0
    return xarray

def evaluateswimmer(Vels,swimname,minx=25,miny=0,maxy=120, savepath=''):
    cycletime = Vels.shape[0]
    total_grid_size_x = Vels.shape[1]
    total_grid_size_y = Vels.shape[2]
    total_grid_size_z = Vels.shape[3]
    
    maxx = total_grid_size_x-minx
    
    #Setup Data Arrays
    U = np.array([])
    V = np.array([])
    uv = np.zeros((cycletime,2))
    for i in range(0,cycletime):
        #Find Sum of Velocities
        u = np.sum(np.mean(Vels[i,minx:maxx,miny:maxy,:,0],axis=-1)-np.mean(Vels[0,minx:maxx,miny:maxy,:,0],axis=-1))
        v = np.sum(np.mean(Vels[i,minx:maxx,miny:maxy,:,1],axis=-1)-np.mean(Vels[0,minx:maxx,miny:maxy,:,1],axis=-1))
        if savepath != '':
            uv[i,0] = u
            uv[i,1] = v
        U = np.append(U,u)
        V = np.append(V,v)
    
    #Drift correct
    t = np.linspace(0,1,U.size)
    U = U - ((U[-1]-U[0])/t[-1])*t
    V = V - ((V[-1]-V[0])/t[-1])*t
    
    if savepath != '':
        #UV drift correct
        uv[:,0] = uv[:,0]-(uv[-1,0]-uv[0,0])*t
        uv[:,1] = uv[:,1]-(uv[-1,1]-uv[0,1])*t
    
    NetU = np.sum(U)
    hc = int(np.round(cycletime/2)) #half cycletime
    EF = np.sum(np.abs(U[:hc]))/(np.sum(np.abs(U[:hc]))+np.sum(np.abs(V[:hc])))
    
    if savepath != '':
        np.savetxt(savepath+swimname+'_UV.txt',uv)
    
    
    return NetU,EF


def SimSwimmer(swimname,slantangle,total_grid_size_x,total_grid_size_y,total_grid_size_z,shape,swimmass=1,visualize=False,savepath=''):
    
    #Build a Velocity Array from Your Objects/ MTFs/Swimmers
    grid = SwimMesh.genGrid(total_grid_size_x,total_grid_size_y,total_grid_size_z,shape)
    
    #Generate Swimmer Base Mesh
    tri,vertdict = SwimMesh.generateMeshHalf(swimname,40,gscale=wscale,arg='pqa1.2')    
    verts = np.array(tri['vertices'])
    triangles = tri['triangles']
    print("Vert Min:")
    print(verts[:,1].min())
    print("Vert Max:")
    print(verts[:,1].max())
    
    
    print('Solving Grid/Material Interactions...')
    #Mindex_build = solve_velocity_grid(verts,triangles,minr,maxr,thic,slantangle,cycletime,profilespline,difspline,grid,shape,streamtime)
    Mindex_build = solve_velocity_grid(verts,triangles,minr,maxr,thic,slantangle,cycletime,profilespline,difspline,grid,shape)
    ObjVels = tf.constant(tf.convert_to_tensor(Mindex_build.astype(np.float16)/ReductionFactor))
    ObjVels = tf.split(ObjVels,cycletime,axis=0)
    
    #Save kinematics summary
    u = np.array([])
    v = np.array([])
    for obj in ObjVels:
        usum =  tf.math.reduce_sum(obj[:,:,:,:,0])
        vsum =  tf.math.reduce_sum(obj[:,:,:,:,1])
        u = np.append(u,usum)
        v = np.append(v,vsum)
    kinUV = np.zeros((2,u.shape[0]))
    kinUV[0,:] = u
    kinUV[1,:] = v
    np.savetxt(savepath+str(swimname)+'_UV_kinematics.txt',kinUV)

    print('Running Simulation...')
    
    #Initialize Video
    savevidcheck = os.path.exists(savepath+'\\CVVideo')
    if savevidcheck == False:
        os.makedirs(savepath+'\\CVVideo')
    vidsavepath = savepath+'\\CVVideo\\'+swimname+'_flow.avi'
    
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    video = cv2.VideoWriter()
    Vsize = int(3)
    if (scalemax!=0):
        success = video.open(vidsavepath, fourcc, 30, (shape[1]*Vsize, shape[0]*Vsize+20), True)
    else:
        success = video.open(vidsavepath, fourcc, 30, (shape[1]*Vsize, shape[0]*Vsize), True)

    #Initialize Grid - Kinematic Viscocity, Grid Shape, Grid Spacing, Time per frame
    totalframes = int(np.round(totalsimtime/dt))
    grid = DBG_LBM.Grid(vis,shape,totalframes,dx=dx,dt=dt/float(streamtime),cycletime=cycletime)
    

    #Material Grid
    grid.MIndex.assign(grid.FlowThrough())
    grid.swimmass = swimmass
    grid.Vel.assign(DBG_LBM.InitialVel(grid.shape)/ReductionFactor)
    grid.ObjVels.assign(ObjVels)


    #Run Program - grid, number of iterations, How often to Save
    #try:
    U,V = DBG_LBM.run(grid,totalframes,video,savenum=savenum,cycletime=cycletime,scalemax=scalemax,streamtime=streamtime,debug=False)
    video.release()
    cv2.destroyAllWindows()

    Vels = grid.ObjVels.numpy()
    Vels = Vels.reshape(cycletime,shape[0],shape[1],shape[2],4)
    Vels = Vels[:,:,:,:,:3]

    RhoScaling = 0.00997*1.02 #g/mm^3
    swimvelocities = grid.OutFlux.numpy()


    velocitycheck = os.path.exists(savepath+'\\SwimVelocities')
    if velocitycheck == False:
        os.makedirs(savepath+'\\SwimVelocities')
    velocitysavepath = savepath+'\\SwimVelocities\\'
    np.savetxt(velocitysavepath+swimname+'_velocity.txt',swimvelocities)

    traveldistance = np.sum(swimvelocities*dt*dx)

    #Save data
    print('Saving Data...')
    hf = h5py.File(savepath+swimname+'.h5', 'w')
    hf.create_dataset('Velocities', data=Vels, compression="gzip", compression_opts=1)
    hf.close()

    if visualize:
        #print('Johnny Needs to program this still...')
        graphFinalStroke(Vels,verts,triangles,swimname,minr,maxr,slantangle,cycletime,profilespline,difspline,savepath,minz=0,zheight=20,figheight = 10,qscale = 10,vlowlim=0,vuplim=5)

    return traveldistance

def solveTriVelocities(Trix,Triy,Triz,xold,yold,zold,dt,frame):
    #Calc New Velocities
    U = np.abs(Trix-xold)/(dt)#*streamtime
    V = np.abs(Triy-yold)/(dt)#*streamtime
    W = np.abs(Triz-zold)/(dt)#*streamtime
    #Fix Sizes
    U = np.concatenate(([0],U))
    V = np.concatenate(([0],V))
    W = np.concatenate(([0],W))

    if frame ==0:
            uSign = np.ones_like(U)
            vSign = -1*np.ones_like(V)
            wSign = np.ones_like(W)
    else:
            #Filters out if each vector is positive or negative
            uSign = binarySign(np.concatenate(([1],Trix-xold)))
            vSign = -1*binarySign(np.concatenate(([1],Triy-yold)))
            wSign = binarySign(np.concatenate(([1],Triz-zold)))

    return U,V,W, uSign,vSign,wSign

#Builds a mask of points containing your swimmer/mtf that aligns with the LBM grid, and assigns them veolcities
def solve_velocity_grid(verts,triangles,minr,maxr,thic,slantangle,cycletime,profilespline,difspline,grid,shape):
    
    #Build a Velocity Array from Your Objects/ MTFs/Swimmers slice by slice
    Mslice =np.zeros((1,shape[0],shape[1],shape[2],4))
    Norms =np.zeros((cycletime,triangles.shape[0],3))

    for i in tqdm(range(cycletime),position=0, leave=True):
        #Finds what percetage of the grid is available for contraction based on a edge pinning
        slantr = SwimMesh.reduce_slant(verts,slantangle)
        
        #Returns coordinate kinematics
        xnew,ynew,znew = SwimMesh.solveContraction(verts,minr,maxr,slantangle,slantr,i,cycletime,profilespline,difspline)

        #Adjust so that MTF isn't sitting on the bottom of the tank or merged with the wall
        znew = znew+2*thic
        ynew = ynew+2*thic

        #Returns Normal Vectors and coordinates of each triangle's centroid
        VertNorms, TriNorm, Trix,Triy,Triz = SwimMesh.ContstructVertNorms(xnew,ynew, znew,triangles,inverse=False)
        if i ==0:
            xold,yold,zold = Trix,Triy,Triz
        
        #Returns velocity Vectors for Each Triangle using position deltas
        U,V,W, Xsign,Ysign,Zsign = solveTriVelocities(Trix,Triy,Triz,xold,yold,zold,dt,i)

        #Returns a list of points that are inside each triangle, labeled by that triangle's number, with positive indicating the topside and negative the bottom side
        zlist = SwimMesh.fast_find_grid(xnew, ynew, znew, triangles, grid, VertNorms,thic=thic)
        zlist = zlist.astype(np.int)

        #Convert to velocities at each triangle point, building a materials interaction grid, slice by slice
        for j in range(0,shape[2]):
            index = np.abs(zlist[j::shape[2]]) #Index of triangles for each z slice
            
            #Assign New Velocities
            zslicex = TriNorm[index,0]*U[index]*Xsign[index]
            zslicey = TriNorm[index,1]*V[index]*Ysign[index]
            zslicez = TriNorm[index,2]*W[index]*Zsign[index]
            
            zslicex = np.transpose(zslicex.reshape(shape[1],shape[0]))
            zslicey = np.transpose(zslicey.reshape(shape[1],shape[0]))
            zslicez = np.transpose(zslicez.reshape(shape[1],shape[0]))
            Mslice[0,:,:,j,0] = zslicex #x velocities
            Mslice[0,:,:,j,1] = zslicey #y velocities
            Mslice[0,:,:,j,2] = zslicez #zvelocities
            Mslice[0,:,:,j,3] = np.transpose(index.reshape(shape[1],shape[0])) #ObjectMask
            Mslice[0,:,:,j,3][Mslice[0,:,:,j,3]!=0]=1 #Set all triangle values to 1

        #print(Mslice.shape)
        if i ==0:
            Mindex_build = Mslice
        else:
            Mindex_build = np.concatenate((Mindex_build,Mslice),axis=0)
            
        #print(TriNorm.shape)
        Norms[i,:,:] = TriNorm
                
        xold,yold,zold = Trix,Triy,Triz
    
    return Mindex_build

def generateDataframe(dnalength):
    #---Make Swimmer Dataframe with Labels ---
    #Takes ~15 min to run for radar plot coordinates

    #Generate List of all possible swimmers
    print('Generating List....')
    dnaAlphabet = SwimNN.SwimDNA.DNABasis()
    fullswimlist = [''.join(i) for i in product(dnaAlphabet, repeat = dnalength)]
    fullswimlist = fullswimlist[1:-1]
    df = pd.DataFrame({'DNA':fullswimlist})
    
    print('Finding Rader Points....')
    alphabetVectors = SwimNN.RadarPlot.constructRadarBasis(SwimNN.SwimDNA.DNABasis())
    RadarPoints = df.DNA.apply(SwimNN.RadarPlot.DNARadarPointsList, args=(alphabetVectors,))
    RadarPoints = pd.DataFrame(RadarPoints.to_list(),columns=['RadX','RadY'])
    df['RadX']=RadarPoints.RadX.to_list()
    df['RadY']=RadarPoints.RadY.to_list()
    
    df['SimNetU'] = np.nan
    df['CDF'] = np.nan
    df['ModelLabel'] = np.nan     
          
    return df


def updateModelLabels(df,model,chunksize=4, dualloss = False):
    splits = int(np.round(df.DNA.size/chunksize))
    breakList = list(SwimNN.SwimSearch.chunks(np.arange(df.DNA.size),splits))
    
    modelLabel = np.zeros(df.DNA.size)
    print('Updating Labels')
    for breaks in breakList:
        print(f'{(breaks[-1]/df.DNA.size)*100:.2f}%...')
        dfchunk = df[breaks[0]:breaks[-1]+1]
        OH_df = SwimNN.NN.OneHotEncodeList(dfchunk)
        if dualloss:
            #modelLabel[breaks[0]:breaks[-1]+1] = model(OH_df[breaks[0]:breaks[-1]+1],training=False)[0].numpy().reshape(-1) #Multiloss
            modelLabel[breaks[0]:breaks[-1]+1] = model(OH_df,training=False)[0].numpy().reshape(-1)
        else:
            modelLabel[breaks[0]:breaks[-1]+1] = model(OH_df,training=False).numpy().reshape(-1)
            #modelLabel[breaks[0]:breaks[-1]+1] = model(OH_df[breaks[0]:breaks[-1]+1],training=False).numpy().reshape(-1)

    return modelLabel

def searchNearestNeighborDE(df,train,simsperloop,loopnum,seed=1337):
    top = df[~df.DNA.isin(train.DNA)].nlargest(int(np.round(simsperloop)),'ModelLabel')
    top['AvgDistance'] =  SwimNN.SwimDNA.avgDnaListDistance(top.DNA)

    mid = top.nlargest(1,'AvgDistance').DNA.to_list()[0]
    far = top.nsmallest(1,'AvgDistance').DNA.to_list()[0]

    DNAlength = len(train.DNA.to_list()[0])

    GenNum = loopnum
    SwimNum = int(np.round(simsperloop/2))-1
    middnalist = GeoSwimmer.GeoSwimmer.GenerateSwimArray(mid,SwimNum,GenNum,DNAlength,EvolveDNA=True,printDATA=False,insert_del=False,double=False,saveDATA = '')
    fardnalist = GeoSwimmer.GeoSwimmer.GenerateSwimArray(far,SwimNum,GenNum,DNAlength,EvolveDNA=True,printDATA=False,insert_del=False,double=False,saveDATA = '')

    #Combine into a DNA list
    dnalist = np.append(middnalist,fardnalist)
    dnalist = np.append(dnalist,far)
    dnalist = np.append(dnalist,mid)
    print(dnalist)
    print("Mid: " + mid)
    print("Far: " + far)
    
    #If already tested, random sample instead
    remain = simsperloop-train[train.DNA.isin(dnalist)].DNA.size #Take random samples for the remaining simulaitons
    if remain>0:
        for Swim in train[train.DNA.isin(dnalist)].DNA.to_list():
            dnalist = dnalist[dnalist!=Swim]
        dnalist = np.append(dnalist,top[~top.DNA.isin(dnalist)].nlargest(remain,'ModelLabel').DNA.to_list())
    return dnalist
    

def checkPrevSimulated(df,SwimList):
    completelist = df[df["SimNetU"].notna()].DNA.to_list()
    print(len(completelist))
    prunelist = list()
    [prunelist.append(x) for x in SwimList if x not in completelist]
    return prunelist

def generateSwimList(df,batchsize,gennum,savepath,dnaLength=6,epochsperloop = 50,seed='1337',lr=0.001,epsilon=1e-07):
    
    tf.keras.backend.clear_session()
    #model = SwimNN.NN.genNN_mixed_model(dnaLength)
    model = SwimNN.NN.Hessian_DualLoss_refine(dnaLength)
    
    loadweights = os.path.exists(savepath+'model.h5')
    if loadweights == False:
        print('No Model Weights found...proceeding with fresh weights') 
    else:
        print('Loading Model weights...')
        model.load_weights(savepath+'model.h5') 
        print('Model Loaded')
    
    
    train = df[df["SimNetU"].notna()]
    print(f'Train min Value: {train["SimNetU"].min()}')
    print(f'Train Max Value: {train["SimNetU"].max()}')
    
    #Estimate CDF of Training Samples
    train_vels = np.array(train["SimNetU"].to_list()) #Convert of Array
    
    print('Fitting CDF to array')
    ge_c,ge_mean,ge_sigma = scipy.stats.genextreme.fit(train_vels) #Fit using extreme value thereom
    print(f'ge_c: {ge_c}, ge_mean: {ge_mean}, ge_sigma: {ge_sigma}')
    train['CDF'] = scipy.stats.genextreme.cdf(train.SimNetU, ge_c,loc=ge_mean,scale=ge_sigma) #-Assigned Guessed CDF
       
    #Generate training inputs for model
    OH_train =tf.cast(SwimNN.NN.OneHotEncodeList(train),tf.float64)
    train_label = tf.convert_to_tensor(train.CDF.to_numpy(),dtype=tf.float64)
    
    #Prep Model
    loss = tf.keras.losses.MeanAbsolutePercentageError()
    model.compile(optimizer=SwimNN.NN.NNOptimizer(lr=lr,epsilon=epsilon),
              loss=(loss,loss),
              metrics=['accuracy'])
    
    #Fit Model to data - Two Labels for Multiloss
    model.fit(OH_train, (train_label,train_label), epochs=epochsperloop)
    
    print('Saving Model Weights...')
    model.save_weights(savepath+'model.h5')
    
    df['ModelLabel'] = updateModelLabels(df,model,chunksize=32,dualloss=True) #Update database's Model Labels
    
    print('Searching top candidates and performing directed evolution')
    #Estimates the most 'central' and 'exterior' DNA sample in the top predicted, and performs directed evolution to generate new swimlist
    SwimList = searchNearestNeighborDE(df,train,batchsize,gennum,seed=seed)
    
    
    
    return SwimList

def colorpallette():
    #New Gray-> Blue Colormap
    cbits = 256
    vals = np.ones((cbits, 4))
    vals[:, 0] = np.linspace(245./256,20./256, cbits)
    vals[:, 1] = np.linspace(245./256,20./256, cbits)
    vals[:, 2] = np.linspace(245./256,120./256,  cbits)
    cpalette = ListedColormap(vals)
    return cpalette

def graphFinalStroke(Vels,verts,triangles,swimmerDNA,minr,maxr,slantangle,cycletime,profilespline,difspline,savepath,minz=0,zheight=25,figheight = 10,qscale = 10,vlowlim=0,vuplim=5,skip=10):
    print("Graphing Results....")  
    xpad = 64*dx
    maxz = int(Vels.shape[3])-1
    minx= 0
    maxx = int(Vels.shape[1])-1
    miny=0
    maxy = int(Vels.shape[2])-1
    
    #Setup Calc- Grid Coordinates
    x = np.linspace(-Vels.shape[1]/2, Vels.shape[1]/2, shape[0])*dx
    y = np.linspace(0, Vels.shape[2], Vels.shape[2])*dx
    z = np.linspace(0, Vels.shape[3], Vels.shape[3])*dx
    xv, yv = np.meshgrid(y,x)
    zxv, zv = np.meshgrid(z,y)

    xz = np.linspace(x.min(),x.max(),Vels.shape[1])
    yz = np.linspace(y.min(),y.max(),Vels.shape[2])
    yzz = np.linspace(z.min(),z.max(),Vels.shape[3])
    xs,ys = np.meshgrid(xz,yz)
    zxvs, zvs = np.meshgrid(yz,yzz)

    midp = int(np.round(Vels.shape[0]/2.0))
    AR = float(xz[Vels.shape[1]-1]-xz[0])/float(yz[Vels.shape[2]-1]-yz[0])
    ARside = float(yzz[maxz]-yzz[minz])/float(yz[Vels.shape[2]-1]-yz[0])
    #sAR = u.shape[1]/float(u.shape[0])

    #Figure Info
    norm = matplotlib.colors.Normalize()
    norm.autoscale(np.array([vlowlim,vuplim]))
    cms = colorpallette()
    sm = matplotlib.cm.ScalarMappable(cmap=cms, norm=norm)
    sm.set_array([])
    
    slantr = SwimMesh.reduce_slant(verts,slantangle)
    for i in range(0,Vels.shape[0]):
        #print(f'{i}/{cycletime}')
        #Get Vectors
        u = scipy.interpolate.griddata((yv.ravel(),xv.ravel()),Vels[i,:,:,zheight,0].ravel(),(xs,ys))
        v = scipy.interpolate.griddata((yv.ravel(),xv.ravel()),Vels[i,:,:,zheight,1].ravel(),(xs,ys))

        xnew,ynew,znew = SwimMesh.solveContraction(verts,minr,maxr,slantangle,slantr,i,cycletime,profilespline,difspline)
        
        #Check Velocity Save path
        check = os.path.isdir(savepath+'\\VelocityTD')
        if not check:
            os.mkdir(savepath+'\\VelocityTD')

        check = os.path.isdir(savepath+'\\VelocityTD\\'+swimmerDNA)
        if not check:
            os.mkdir(savepath+'\\VelocityTD\\'+swimmerDNA)
        
        M = np.sqrt(u**2+v**2)

        ##Plot Velocity Top Down##
        fig, ax = subplots(figsize=(figheight*AR,figheight))
        ax.pcolormesh(xs,ys, M,cmap=cms,vmin=vlowlim,vmax=vuplim)

        for vertices in triangles:
            ax.plot(xnew[vertices],ynew[vertices],'r--',linewidth=3)
            ax.plot(xnew[[vertices[-1],vertices[0]]],ynew[[vertices[-1],vertices[0]]],'r--',linewidth=3)

        #q = ax.quiver(xs[::skip,::skip],ys[::skip,::skip],u[::skip,::skip],v[::skip,::skip],color='k',clim=(vlowlim,vuplim),width=0.008,scale=3,alpha=0.9)
        q = ax.quiver(xs[::skip,::skip],ys[::skip,::skip],u[::skip,::skip],v[::skip,::skip],color='k',clim=(vlowlim,vuplim),width=0.008,scale=qscale,alpha=0.9)
        ax.set_xlabel('X (mm)',size=30,labelpad=20)
        ax.set_ylabel('Y (mm)',size=30,labelpad=20)
        cbar = fig.colorbar(sm)
        cbar.set_label(('Velocity (mm/s)'), rotation=270,fontsize=30,labelpad=30)
        xlim(xz[minx],xz[maxx])
        ylim(yz[miny],yz[maxy])
        savefig(savepath+'\\VelocityTD\\'+swimmerDNA+'\\'+str(i)+'.png')
        clf()
        close()
        
    return

In [ ]:
#Initial Variables
#Area constraints (mm)
total_grid_size_x = 18.0 #mm
total_grid_size_y = 12.0 #mm
total_grid_size_z = 12.0 #mm


ReductionFactor = 1 #Size scaling factor, default 1
streamtime = 10 #Frames of liquid streaming per frame of object movement

#Viscosity
Re = 60 #Reynolds Number
maxVel = 21 #mm/s
swimmass = 1 #mass relative to density of fluid

#Dependent Variable Setup
Re = Re*ReductionFactor
vis = 1*maxVel*2/(Re) #Rho*Vel*CharcLength/Renolds
vis = vis*1.02 #Change based on Tyrode's estimated viscocity
swimmass = swimmass*ReductionFactor

#Time Steps
dt = 1.0/60.0 #Second per step
dx = 0.125#0.075 #Size of each grid space
totalsimtime = 24.0 #in seconds
savenum = 4 #Save every number of n frames

#Image Scalling - Set to zero to remove frame
scalemax = 4 #mm/s
scalemax = scalemax/ReductionFactor

#Stimulation
Hz = 1.0
cycletime = int(np.round(1 /(Hz*dt)))
print('Cycle frames: '+ str(cycletime))

#Mtf Variables - 
thic = 2*dx#.1 #Thickness of Film
slantangle = -30.0 #Angle that the film is patterned at
maxr = 8  #Circle max - mm
minr = 1.4 #Circle min - mm


#Scaling Factor for Swimmers - Default = 5 for ~ 5 mm max width swimmers
wscale = 4

#Unit Conversion (pica to mm)
picatoMM = 6.4/1.696 #mm/pica


In [ ]:
#Cleanup From Constants
xsize = np.round(total_grid_size_x/(float(dx)*2))*2
ysize = np.round(total_grid_size_y/(float(dx)*2))*2
zsize = np.round(total_grid_size_z/(float(dx)*2))*2
shape = np.array([int(xsize), int(ysize), int(zsize)])

print('grid points: ' + str(shape))

profile = np.loadtxt('Contraction_profile2.txt') # Contraction profile from MTF kymograph, determining film bending rate

#Spline fitting of contraction profile to make it a continuose function
t = np.linspace(0,1,profile.size)
profilespline = UnivariateSpline(t,profile,k=3,s=.001)

#Derivative of Change - determing transition from angled to flat MTFs
t = np.linspace(0,10,30)
a,b = 3,-0.2
g = scipy.stats.gamma(a,b).pdf(t)
g = g/g.max()
g = (1-g)*0.58 #0.58 is based on relative angled cantilever contraction
dif = g

difspline = UnivariateSpline(np.linspace(0,1,dif.size),dif,k=3,s=.001)
print('Contraction Profile Loaded...')



#print('Swim Width: '+ str(np.max(verts[:,0])-np.min(verts[:,0])))
#print('Swim Height: '+ str((np.max(verts[:,1])-np.min(verts[:,1]))*2))

In [ ]:
#Mass Run of Angles

#Initial Swimmer Simulation Conditions
SeedDNA = 'DBGDBG' #Random seed to initialize which inputs to initially search


savepath = 'G:\\....\\' #Filepath of where to save the outputs. Should end in a folder location denoted by '\\'
initialbatch = 200 #How many swimmers to train on in the initial search
batchsize = 50#24
generations = 20 #Number of generations to search
BaseLength = 6 #number of sDNA basis functions to include in a swimmer
slantangle = -30 #Fin print angle
epochsperloop = 50
Visualize = True #Save visualization data for each sDNA sequence


#Record Swimmers Completed
completecheck = os.path.exists(savepath+'results.pkl')
if completecheck == False:
    print('Generating Data frame structure (Takes ~15 min)')
    df = generateDataframe(BaseLength)
    df.to_pickle(savepath+'results_new.pkl')
else:
    print('Loading Dataframe...')
    df = pd.read_pickle(savepath+'results.pkl')
    print('Dataframe Loaded')

gentest = os.path.exists(savepath+'currentgen.txt')
if gentest == False:
    currentgen = np.zeros(1,dtype=int)
    #currentgen = int(currentgen)
    np.savetxt(savepath+"currentgen.txt",currentgen)
    currentgen = 0

else:
    currentgen = int(np.genfromtxt(savepath+"currentgen.txt").tolist())
print(f'Current Generation:{currentgen}')
    
#Main loop- Running For Swimmer Data    
print(currentgen+generations)
for gennum in range(currentgen,currentgen+generations):
    print(f'Running Generation: {gennum}')
    
    SwimList = list('')
    
    if os.path.exists(savepath+'CurrentSwimlist.txt'):
        SwimList = np.loadtxt(savepath+'CurrentSwimlist.txt',dtype=np.str)
    
    SwimList = checkPrevSimulated(df,SwimList)
    
    if len(SwimList)<= 0:
        #Prepare list of swimmers to simulate
        if gennum == 0:
            seededlist = SwimNN.SwimSearch.seededList(df)
            lenSeedList = len(seededlist.index)
            if initialbatch>lenSeedList:
                SwimList = seededlist.DNA.to_list()
            else:
                SwimList = list('')

            remaining = initialbatch-len(SwimList)
            seedint = GeoSwimmer.GeoSwimmer.seedtoInt(SeedDNA)%2**30
            SwimList = SwimList+ df.sample(remaining,random_state=seedint).DNA.to_list()
            print('Initial Sample')
        else:
            seedint = (GeoSwimmer.GeoSwimmer.seedtoInt(SeedDNA)+(gennum*batchsize))%2**30
            SwimList = generateSwimList(df,batchsize,gennum,savepath,dnaLength=BaseLength,epochsperloop = epochsperloop,seed=seedint)
            np.savetxt(savepath+"currentgen.txt",np.array([gennum]))
            
            #SaveRadarPlot
            check = os.path.isdir(savepath+'\\RadarPlot')
            if not check:
                os.mkdir(savepath+'\\RadarPlot')

            sample = df.sample(20000,random_state=seedint)
            fig = SwimNN.RadarPlot.PlotRadarMesh(sample.RadX,sample.RadY,sample.ModelLabel,figheight=10,figwidth=10)
            savefig(savepath + "\\RadarPlot\\"+str(df[df["SimNetU"].notna()].DNA.size)+".png")
            

            
            print(f'NewSwimmers: {SwimList}')
    
    print(SwimList)
    #Test If you have run swimmers before
    SwimList = checkPrevSimulated(df,SwimList)
    np.savetxt(savepath+'CurrentSwimlist.txt',SwimList,fmt='%s')

    
    #Simulate Swimmers
    for swimname in SwimList:
        print('Running: '+str(swimname))
        swimnum = df[df["SimNetU"].notna()].DNA.size
        print(f'SwimmerNum: {swimnum}')
        tf.compat.v2.keras.backend.clear_session()

        distancetravelled = SimSwimmer(swimname,slantangle,total_grid_size_x,total_grid_size_y,total_grid_size_z,shape,swimmass=swimmass,visualize=Visualize,savepath=savepath)
        print(f'Swimmer Travel Distance: {distancetravelled}')
        
        df.loc[df[df.DNA.isin([swimname])].index,"SimNetU"] = distancetravelled
        df.loc[df[df.DNA.isin([swimname])].index,"GenNum"] = gennum
        df.loc[df[df.DNA.isin([swimname])].index,"time"] = time.time()
        df.to_pickle(savepath+'results_updated.pkl')
        if (swimnum%100 == 0):
            df.to_pickle(savepath+'results_'+str(swimnum)+'.pkl')


In [ ]:

#Simulate a Single sDNA Sequence
swimname = 'GLIDAA'

#Filepath of where to save the outputs. Should end in a folder location denoted by '\\'
savepath = 'G:\\...\\'

slantangle = -30
Visualize = True

distancetravelled = SimSwimmer(swimname,slantangle,total_grid_size_x,total_grid_size_y,total_grid_size_z,shape,swimmass=swimmass,visualize=Visualize,savepath=savepath)

In [ ]:
def applyCustomColorMap(im_gray) :
    cbits = 256
    lut = np.zeros((256, 1, 3), dtype=np.uint8)

    #Red
    lut[:, 0, 0] = np.linspace(245,20, cbits, dtype=np.uint8)[:]
    #Green
    lut[:, 0, 1] = np.linspace(245,20, cbits, dtype=np.uint8)[:]

    #Blue
    lut[:, 0, 2] = np.linspace(245,120,  cbits, dtype=np.uint8)[:]
    print(lut)
    #Apply custom colormap through LUT
    im_color = cv2.LUT(im_gray, lut)
    
    return im_color;

In [ ]:
def colorpallette():
    #New Gray-> Blue Colormap
    cbits = 256
    vals = np.ones((cbits, 4))
    vals[:, 0] = np.linspace(245./256,20./256, cbits)
    vals[:, 1] = np.linspace(245./256,20./256, cbits)
    vals[:, 2] = np.linspace(245./256,120./256,  cbits)
    cpalette = ListedColormap(vals)
    return cpalette

In [ ]:
def apply_custom_colormap(image_gray):

    assert image_gray.dtype == np.uint8, 'must be np.uint8 image'
    if image_gray.ndim == 3: image_gray = image_gray.squeeze(-1)

    # Initialize the matplotlib color map
    cms = colorpallette()
    sm = matplotlib.cm.ScalarMappable(cmap=cms)

    # Obtain linear color range
    color_range = sm.to_rgba(np.linspace(0, 1, 256))[:,0:3]    # color range RGBA => RGB
    color_range = (color_range*255.0).astype(np.uint8)         # [0,1] => [0,255]
    color_range = np.squeeze(np.dstack([color_range[:,2], color_range[:,1], color_range[:,0]]), 0)  # RGB => BGR

    # Apply colormap for each channel individually
    channels = [cv2.LUT(image_gray, color_range[:,i]) for i in range(3)]
    return np.dstack(channels)